In [1]:
import sys
import time
import json

sys.path.append('../core/python')

from commons import inc_inference_with_model, show_heatmap
from imagenet_classes import class_names
from vgg16 import VGG16
from resnet18 import ResNet18
from inception3 import Inception3

In [2]:
image_file_path = "../images/input/imagenet/dog.jpg"

In [3]:
vgg_model = VGG16(beta=1.0, gpu=True, n_labels=1000).eval()

/usr/local/lib/python2.7/dist-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [8]:
begin_time = time.time()
_,_,_ = inc_inference_with_model(vgg_model, image_file_path, patch_size=4, stride=4, beta=1.0, x0=0, y0=0, image_size=224, x_size=224,
                      y_size=224, version='v1', gpu=True, c=0.0)
end_time = time.time()
time_elapsed = end_time - begin_time

In [9]:
print(time_elapsed)

2.34310388565


In [ ]:
'''
start the experimeng with vgg16
iterate all time-possibilities of vgg16
store all the patch,stride,height,width possibilities into .json 
as the backend for later estimate time cost
of running Krypton
'''
patch_dict = [1,2,4,8,16]
stride_dict = [1,2,4,8,16]

data = {}
data['vgg16'] = []

for pt in patch_dict:
    print('pt: ' + str(pt))
    for st in stride_dict:
        print('st: ' + str(st))
        for x in range(1,225):
            print('x: ' + str(x))
            for y in range(1,225):
                #print('y: ' + str(y))
                begin_time = time.time()
                _,_,_ = inc_inference_with_model(vgg_model, image_file_path, patch_size=pt, stride=st, beta=1.0, x0=0, y0=0, image_size=224, x_size=x,
                                      y_size=y, version='v1', gpu=True, c=0.0)
                end_time = time.time()
                time_elapsed = end_time - begin_time
                data['vgg16'].append({
                    'patch': pt,
                    'stride': st,
                    'width': x,
                    'height': y,
                    'time': time_elapsed
                })
        
        
with open('time-estimation.txt', 'w') as outfile:
    json.dump(data, outfile)

pt: 1
st: 1
x: 1
x: 2
x: 3
x: 4
x: 5
x: 6
x: 7
x: 8
